In [23]:
import gurobipy as gp
from gurobipy import GRB

def solve_linear_program(n, E, k, color_constraints, color_sets):
    # 创建一个模型
    model = gp.Model("max_independent_set")

    # 添加变量
    x = model.addVars(n, vtype=GRB.BINARY, name="x")

    # 目标函数 (3.1)
    model.setObjective(gp.quicksum(x[i] for i in range(n)), GRB.MAXIMIZE)

    # 约束 (3.2) 相邻节点不能同时被选中
    for (i, j) in E:
        model.addConstr(x[i] + x[j] <= 1, f"edge_{i}_{j}")

    # 约束 (3.3) 选中的节点数不能超过 k
    model.addConstr(gp.quicksum(x[i] for i in range(n)) <= k, "max_selection")

    # 约束 (3.4) 颜色类的上下界约束
    for c, (l_c, h_c) in color_constraints.items():
        model.addConstr(gp.quicksum(x[i] for i in color_sets[c]) >= l_c, f"lb_color_{c}")
        model.addConstr(gp.quicksum(x[i] for i in color_sets[c]) <= h_c, f"ub_color_{c}")

    # 约束 (3.5) x_i 是二元变量
    # 这已经在定义变量时使用 vtype=GRB.BINARY 自动包含了。

    # 求解模型
    model.optimize()

    # 输出结果
    if model.status == GRB.OPTIMAL:
        selected_nodes = [i for i in range(n) if x[i].x > 0.5]
        print(f"Optimal solution: {selected_nodes}")
    else:
        print("No optimal solution found.")

# 示例参数
n = 10  # 节点数量
E = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]  # 边集
k = 5  # 最多选中5个节点
color_constraints = {0: (1, 3), 1: (1, 3)}  # 每个颜色类的上下界
color_sets = {0: [0, 1, 2, 3, 4], 1: [5, 6, 7, 8, 9]}  # 节点的颜色分配

# 求解线性规划
solve_linear_program(n, E, k, color_constraints, color_sets)

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10200H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 10 columns and 48 nonzeros
Model fingerprint: 0x61175e3e
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 14 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%
Optimal solution: [0, 2, 4, 7, 9]
